# Recommendation Systems

![Alt text](Images\Becauseyouwatched.jpg)

## BUSINESS UNDERSTANDING.

## Business Overview.

Imagine the number of streaming sites, now imagine the challenges the platform faces: the two main questions being how do they get to help  users discover content they will not only love but also will keep them engaged over time- remember there are thousands of titles available as such users could experience decision fatigue which may lead to lower watch time and or increased churn. Now how about having a well-designed recommendation system that can transform this experience by only surfacing relevant and personalized content that resonates with one's individual taste and preference?

This project is therefore designed to build a movie recommendation engine using the MovieLens dataset, with the aim of improving the retention of users through personalized suggestions and we shall do so by analyzing the user ratings and preferences. In the end we shall be able to deliver top five movie recommendations that feel intuitive, relevant and engaging.

## Problem statement.

As we have seen that its really a struggle by users to find content that matches their prefences, especially as more movies gets produced each day. Therefore our question is ***'how can we deliver personalized movie recommendations that will ultimately increase user satsifaction and retention on a streaming platform?'***

## Stakeholders

**Product team**- to improve user engagement & retention through personalization  
**Data Scientist**- For building and validating recommendation engine  
**Marketing team**- for segmentation and promotion of contetnt based on preferences  
**Owners of streaming platforms/ Executive leadership**- Evaluation of ROI of a well designed personalized system on platform performance.


## Success Metrics.
- Building a model that generates top five movie recommendations per user.
- Improvement of user engagement as a result of tailor-making contetnt to user preferences.
- Provision of actionable insights for product and marketing team to use.
- Visualizing resulsts for non technical team e.g the executive.


## Type of Recommendation and Model Evaluation Metrics.
This project will be focusing on personalized recommendations, leveraging collaborative filtering to unearth latent user preferances and as a result suggest movies aligned with their taste. Some of the metrics that we may deploy include:

- **RMSE/MAE** -For rating prediction accuracy
- **Precision, Recall, F1 Score** - For quality ranking.
- **Coverage, Diversity** - To help with assessment of recomendation variety & system robustness.

In [ ]:
# Eric lead developing here | Lynn, feel free to support co-creation 
# Address Business overview, problem statement, stakholders, goals/objectives (e.g., product suggestions, movie recommendations, personalized content))
# Determine the type of recommendation (personalized vs. non-personalized)
# Outline some metrics we may want to use in model evaluation-

## Data Understanding  

In [ ]:
# Load the libraries


In [ ]:
# Load the datasets


In [ ]:
# Newton work on the Data Understanding part
# Briefly describe each dataset in the CSV format

## Data Preparation

In [ ]:
# Joackim kindly lead here : Check need to create a single dataframe with desired variables including justifications as part of notes


## Modeling

In [ ]:
# Perform data preprocessing  
# Clean the data to handle missing values, duplicates and outliers
# Normalize or scale numerical features if applicable
# One-hot encode categorical data to suitable formats
# Split into training and test sets
# Choose a recommendation approach ans apply
# Build a model using an algorithm of choice - KNN, SVD or deep learning
# Train model using hitorical interaction data
# optimize the hyperparameters

## Evaluation 

In [ ]:
# Use metrics such as: 
#RMSE/MAE for Rating predictions
# Precision/Recall/F1 score - For ranking 
# MAP/NDCG for ordered recommendations

## Findings and Conclusion

In [ ]:
# Use the information above to answer the objectives outlined in the introduction

## Recommendations

In [ ]:
# Make recommendations based on the findings and interest of the stakeholders